# STATIC GRAPH

In [ ]:
import tensorflow as tf
from tensorflow.contrib.layers import xavier_initializer as xinit

## TODO

- [x] Mask padding while calculating loss
- [x] Deal with variable length input sequence
- [ ] Add `EOS` (end of sentence) token to data (target sequence)

## DATA

In [ ]:
import data_utils
metadata, idx_q, idx_a = data_utils.load_data('../data/')

In [ ]:
# add special symbol
i2w = metadata['idx2w'] + ['GO'] + ['EOS']
w2i = metadata['w2idx']
w2i['GO'] = len(i2w)-2
w2i['EOS'] = len(i2w)-1

## Parameters

In [ ]:
B = 8
L = len(idx_q[0])
vocab_size = len(i2w)
enc_hdim = 250
dec_hdim = enc_hdim

## Graph

In [ ]:
tf.reset_default_graph()

### Placeholders

In [ ]:
inputs = tf.placeholder(tf.int32, shape=[B,L], name='inputs')
targets = tf.placeholder(tf.int32, shape=[B,L], name='targets')
decoder_inputs = tf.concat(
    values=[tf.constant(w2i['GO'], dtype=tf.int32, shape=[B,1]), targets[:, 1:]],
    axis=1)
training = tf.placeholder(tf.bool, name='is_training')

In [ ]:
decoder_inputs, targets, inputs

## Sequence Length

In [ ]:
# Encoder
#  Get sequence lengths
def seq_len(t):
    return tf.reduce_sum(tf.cast(t>0, tf.int32), axis=1)

enc_seq_len = seq_len(inputs)

In [ ]:
# Decoder
#   Mask targets
#    to be applied to cross entropy loss
padding_mask = tf.cast(targets > 0, tf.float32 )

## Embedding

In [ ]:
emb_mat = tf.get_variable('emb', shape=[vocab_size, enc_hdim], dtype=tf.float32, 
                         initializer=xinit())
emb_enc_inputs = tf.nn.embedding_lookup(emb_mat, inputs)
emb_dec_inputs = tf.nn.embedding_lookup(emb_mat, decoder_inputs)

## Encoder

In [ ]:
with tf.variable_scope('encoder'):
    encoder_cell = tf.contrib.rnn.LSTMCell(num_units=enc_hdim)
    encoder_outputs, context = tf.nn.dynamic_rnn(cell=encoder_cell, # encoder cell
                          inputs=emb_enc_inputs, # embedding input sequences
                          sequence_length=enc_seq_len, # lengths of sequences
                          initial_state = # zero state of cell
                                encoder_cell.zero_state(batch_size=B, dtype=tf.float32)
                         )

## Decoder

In [ ]:
# inference - custom rnn
def inference_decoder(cell, state):
    #tf.get_variable_scope().reuse_variables()
    with tf.variable_scope('decoder_1'):
        dec_outputs = []
        input_ = tf.constant(w2i['GO'], dtype=tf.int32, shape=[B,])
        input_ = tf.nn.embedding_lookup(emb_mat, input_)
        
        for i in range(L):
            decoder_output, state = cell(input_, state)
            dec_outputs.append(decoder_output)
            input_ = decoder_output

    return tf.stack(dec_outputs), None

In [ ]:
with tf.variable_scope('decoder') as scope:
    decoder_cell = tf.contrib.rnn.LSTMCell(num_units=dec_hdim)
    
    decoder_outputs, _ = tf.nn.dynamic_rnn(cell=decoder_cell,
                          inputs=emb_dec_inputs, 
                          initial_state= context,
                          scope='decoder_1')
                         
    tf.get_variable_scope().reuse_variables()
    
    decoder_outputs_inf, _ = inference_decoder(decoder_cell, context)

## Logits and Probabilities

In [ ]:
Wo = tf.get_variable('Wo', shape=[dec_hdim, vocab_size], dtype=tf.float32, 
                         initializer=xinit())
bo = tf.get_variable('bo', shape=[vocab_size], dtype=tf.float32, 
                         initializer=xinit())
proj_outputs = tf.matmul(tf.reshape(decoder_outputs, [B*L, dec_hdim]), Wo) + bo
proj_outputs_inf = tf.matmul(tf.reshape(decoder_outputs_inf, [B*L, dec_hdim]), Wo) + bo

logits = tf.cond(tf.random_normal(shape=()) > 0.,
    lambda : tf.reshape(proj_outputs, [B, L, vocab_size]),
    lambda : tf.reshape(proj_outputs_inf, [B, L, vocab_size])
                )
probs = tf.nn.softmax(tf.reshape(proj_outputs_inf, [B, L, vocab_size]))

## Loss

In [ ]:
cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
    logits =  logits,
    labels = targets)
# apply mask
masked_cross_entropy = cross_entropy * padding_mask
# average across sequence, batch
loss = tf.reduce_mean(cross_entropy)

## Optimizatin

In [ ]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(loss)

## Inference

In [ ]:
prediction = tf.argmax(probs, axis=-1)

# TRAINING

In [ ]:
config = tf.ConfigProto(allow_soft_placement = True)
sess = tf.InteractiveSession(config = config)
sess.run(tf.global_variables_initializer())

## Training parameters

In [ ]:
num_epochs = 20

## Start Training

In [ ]:
for i in range(num_epochs):
    avg_loss = 0.
    for j in range(len(idx_q)//B):
        _, loss_v = sess.run([train_op, loss], feed_dict = {
            inputs : idx_q[j*B:(j+1)*B],
            targets : idx_a[j*B:(j+1)*B]
        })
        avg_loss += loss_v
        if j and j%30==0:
            print('{}.{} : {}'.format(i,j,avg_loss/30))
            avg_loss = 0.

### Compare CE and masked CE

In [ ]:
ce, mce = sess.run([cross_entropy, masked_cross_entropy], feed_dict = {
            inputs : idx_q[j*B:(j+1)*B],
            targets : idx_a[j*B:(j+1)*B]
        })

In [ ]:
ce[0], mce[0]

## Test inference

In [ ]:
j = 117
pred_v = sess.run(prediction, feed_dict = {
            inputs : idx_q[j*B:(j+1)*B],
            #targets : idx_a[j*B:(j+1)*B]
        })

In [ ]:
pred_v[0], idx_a[j*B:(j+1)*B][0]

In [ ]:
def arr2sent(arr):
    return ' '.join([i2w[item] for item in arr])

In [ ]:
for i in range(B):
    print(arr2sent(pred_v[i]))

In [ ]:
arr2sent( idx_a[j*B:(j+1)*B][11])